# SmartLife Concierge Agent – FIXED Google Colab Version
This notebook is fully corrected and ready to run.

In [ ]:
!pip install --quiet google-generativeai PyPDF2

In [ ]:

import google.generativeai as genai
from PyPDF2 import PdfReader
import datetime
from typing import List, Dict, Optional

# ✅ Add your Gemini API key here
GEMINI_API_KEY = "YOUR_API_KEY_HERE"   # <-- replace only this string

genai.configure(api_key=GEMINI_API_KEY)

MODEL_NAME = "gemini-1.5-flash"


In [ ]:

def call_llm(system_prompt: str, user_prompt: str) -> str:
    model = genai.GenerativeModel(MODEL_NAME)
    response = model.generate_content(system_prompt + "\n\nUser:\n" + user_prompt)
    return response.text.strip()


In [ ]:

def extract_text_from_pdf(path, max_pages=5):
    reader = PdfReader(path)
    text = ""
    for p in reader.pages[:max_pages]:
        t = p.extract_text()
        if t: text += t + "\n"
    return text


In [ ]:

class SmartLifeAgent:
    def __init__(self):
        self.tasks=[]
        self.counter=1
        self.base="You are SmartLife Concierge Agent. Respond clearly."

    def add_task(self,title,due="",notes=""):
        t={"id":self.counter,"title":title,"due":due,"notes":notes,"done":False}
        self.tasks.append(t); self.counter+=1; return t

    def list_tasks(self,pending=False):
        return [t for t in self.tasks if not t["done"]] if pending else self.tasks

    def complete(self,tid):
        for t in self.tasks:
            if t["id"]==tid: t["done"]=True; return True
        return False

    def summarize_pdf(self, path):
        return call_llm(self.base, extract_text_from_pdf(path))

    def summarize_text(self, text):
        return call_llm(self.base, "Summarize:\n"+text)

    def explain_code(self, code, lang="C"):
        return call_llm(self.base, f"Explain {lang} code:\n{code}")

    def study_plan(self, subs,date,hours):
        return call_llm(self.base, f"Study plan for {subs}, exam {date}, {hours} hours/day")

    def chat(self,msg):
        return call_llm(self.base,msg)


In [ ]:

agent = SmartLifeAgent()

def chat_loop():
    print("🤖 SmartLife Agent Ready! Type /help.")
    helptext="""Commands:
/help
/chat <msg>
/add_task
/list_tasks
/list_open
/complete <id>
/summarize_text
/summarize_pdf
/explain_code
/study_plan
"""
    while True:
        msg=input("You > ").strip()
        if msg=="exit": print("Bye!"); break
        if msg=="/help": print(helptext)
        elif msg.startswith("/chat"): print(agent.chat(msg.replace("/chat","")))
        elif msg=="/add_task":
            print(agent.add_task(input("Title:"),input("Due:"),input("Notes:")))
        elif msg=="/list_tasks": print(agent.list_tasks())
        elif msg=="/list_open": print(agent.list_tasks(True))
        elif msg.startswith("/complete"): print(agent.complete(int(msg.split()[1])))
        elif msg=="/summarize_text":
            print("Paste text, END to finish"); lines=[]
            while True:
                l=input(); 
                if l=="END": break
                lines.append(l)
            print(agent.summarize_text("\n".join(lines)))
        elif msg=="/summarize_pdf": print(agent.summarize_pdf(input("PDF path:")))
        elif msg=="/explain_code":
            lang=input("Lang:"); print("Paste code, END to finish"); lines=[]
            while True:
                l=input()
                if l=="END": break
                lines.append(l)
            print(agent.explain_code("\n".join(lines),lang))
        elif msg=="/study_plan":
            print(agent.study_plan(input("Subjects:"),input("Exam Date:"),input("Hours:")))
        else: print("Unknown. /help for commands.")


In [ ]:
# Run the agent:
# chat_loop()